In [85]:
import json
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [108]:
read = [['2020-08-01', 'abc Pflegeheim Altenpfleger Pflegeheim jhk'], ['2024-06-07', 'das the pflegeheim Altenpflegefachkraft Pflegeheim zupa cfg'], ['2025-01-03', 'abc Altenpflegerin pflegeheim abc'], ["2024-01-01", "abcd Pflegefachperson ambulant Pflegedienst123 cdg"], ['04.07.2025', '123 die Krankenpflegerin innere krankenhaus abc'], ["01.01.2025 17:00:43", "abcd ambulanter pflegedienst Pflegefachmann 123"], ["12/31/2023", "krankenhaus innere Krankenschwester 12acde"], ["2023-02-01", "abcd ambulante Pflegedienst Pflegefachfrau 74"], ["2024-02-02", "Krankenhaus innere Krankenpfleger abcd"], ['2024-03-02', 'a Kinderkrankenpfleger b']]
readpro = []
for i in range(len(read)):
    readpro.append(read[i][1])

In [110]:
stop_words = stopwords.words('german')
# SteA laden aus json
# LDA drüberlaufen lassen (mallet)

In [112]:
"""with open('Pflege_2016.json', 'r') as file:
    output = json.load(file)
readpro = [[value] for value in output["full_text"].values()]"""

'with open(\'Pflege_2016.json\', \'r\') as file:\n    output = json.load(file)\nreadpro = [[value] for value in output["full_text"].values()]'

In [130]:
def tokenize(text):
    # Remove punctuation and non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Split into words
    tokens = text.split()
    return tokens
prepro = []
for value in readpro:
    add = tokenize(value)
    prepro.append(add)
# Build the bigram and trigram models
bigram = gensim.models.Phrases(prepro, min_count=2, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[prepro], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['abc', 'altenpfleger', 'pflegeheim', 'jhk']


In [126]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [138]:
# Remove Stop Words
data_words_nostops = remove_stopwords(prepro)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('de_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pflegeheim', 'altenpfleg', 'Pflegeheim', 'jhk']]


In [140]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [152]:
"""We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes."""
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

[(0,
  '0.199*"innerer" + 0.119*"krankenhaus" + 0.119*"acde" + 0.119*"Krankenhaus" '
  '+ 0.118*"Abcd" + 0.118*"cdg" + 0.044*"Altenpflegerin" + 0.041*"pflegeheim" '
  '+ 0.041*"jhk" + 0.041*"Pflegeheim"'),
 (1,
  '0.142*"altenpfleg" + 0.142*"Pflegeheim" + 0.142*"jhk" + 0.142*"pflegeheim" '
  '+ 0.138*"Altenpflegerin" + 0.050*"cdg" + 0.050*"Abcd" + 0.050*"innerer" + '
  '0.049*"Krankenhaus" + 0.049*"acde"')]

Perplexity:  -3.07256592825676

Coherence Score:  0.4471578578396118


/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=15856) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=15856) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=15856) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=15856) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.066267  0.0       1        1  56.876822
1     -0.066267  0.0       2        1  43.123178, topic_info=              Term      Freq     Total Category  logprob  loglift
1       altenpfleg  1.000000  1.000000  Default  11.0000  11.0000
0       Pflegeheim  1.000000  1.000000  Default  10.0000  10.0000
2              jhk  1.000000  1.000000  Default   9.0000   9.0000
3       pflegeheim  1.000000  1.000000  Default   8.0000   8.0000
4   Altenpflegerin  1.000000  1.000000  Default   7.0000   7.0000
8          innerer  1.000000  1.000000  Default   6.0000   6.0000
10     krankenhaus  1.000000  1.000000  Default   5.0000   5.0000
9             acde  1.000000  1.000000  Default   4.0000   4.0000
7      Krankenhaus  1.000000  1.000000  Default   3.0000   3.0000
5             Abcd  1.000000  1.000000  Default   2.0000   2.0000
6              cdg  1.000000  1.000000  Default   1.0000   1.0000
8          innerer  1.355909  1.612520   Topic1  -1.6162   0.3910
10     krankenhaus  0.813164  1.065792   Topic1  -2.1274   0.2937
9             acde  0.812142  1.065688   Topic1  -2.1287   0.2926
7      Krankenhaus  0.811748  1.065648   Topic1  -2.1292   0.2921
5             Abcd  0.808364  1.065304   Topic1  -2.1334   0.2883
6              cdg  0.807679  1.065235   Topic1  -2.1342   0.2875
4   Altenpflegerin  0.300298  1.013695   Topic1  -3.1236  -0.6523
3       pflegeheim  0.279138  1.011546   Topic1  -3.1967  -0.7232
2              jhk  0.278978  1.011529   Topic1  -3.1972  -0.7238
0       Pflegeheim  0.278941  1.011526   Topic1  -3.1974  -0.7239
1       altenpfleg  0.278858  1.011517   Topic1  -3.1977  -0.7242
1       altenpfleg  0.732659  1.011517   Topic2  -1.9549   0.5186
0       Pflegeheim  0.732585  1.011526   Topic2  -1.9550   0.5185
2              jhk  0.732551  1.011529   Topic2  -1.9550   0.5184
3       pflegeheim  0.732408  1.011546   Topic2  -1.9552   0.5182
4   Altenpflegerin  0.713397  1.013695   Topic2  -1.9815   0.4898
6              cdg  0.257556  1.065235   Topic2  -3.0003  -0.5786
5             Abcd  0.256940  1.065304   Topic2  -3.0027  -0.5811
7      Krankenhaus  0.253900  1.065648   Topic2  -3.0146  -0.5933
9             acde  0.253546  1.065688   Topic2  -3.0160  -0.5947
10     krankenhaus  0.252628  1.065792   Topic2  -3.0196  -0.5984
8          innerer  0.256611  1.612520   Topic2  -3.0040  -0.9969, token_table=      Topic      Freq            Term
term                                 
5         1  0.938699            Abcd
4         2  0.986490  Altenpflegerin
7         1  0.938396     Krankenhaus
0         2  0.988606      Pflegeheim
9         1  0.938361            acde
1         2  0.988614      altenpfleg
6         1  0.938760             cdg
8         1  0.620147         innerer
2         2  0.988602             jhk
10        1  0.938269     krankenhaus
3         2  0.988586      pflegeheim, R=11, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])